# Intelligent Realestate Investing 

In [244]:
#https://rapidapi.com/apidojo/api/realtor
import pandas as pd
import requests
import json #helps manipulate json
import csv # helps with csv files
from bs4 import BeautifulSoup
!pip install pyzillow

from pyzillow.pyzillow import ZillowWrapper, GetDeepSearchResults

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import os
import shutil

# for web driver down below
email = 'blaketindol@gmail.com'
passwordz = YOU KNOW
!pip install folium
import folium # look in china

#https://www.melissa.com/v2/lookups/addresssearch/?number=&street=Carino+Ct&city=Paw+Paw&state=MI&zip=49079-9661&freeForm=

You should consider upgrading via the 'c:\users\btindol\anaconda3\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\btindol\anaconda3\python.exe -m pip install --upgrade pip' command.


In [56]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [4]:
# Function using the realator api to grabe houses that are for sale!!

def api_property_list_for_sale(city, state, prop_type, limit=200):

    url = "https://realtor.p.rapidapi.com/properties/v2/list-for-sale"

    querystring = {"city":city,"limit":"200","offset":"0","state_code":state,"sort":"relevance","prop_type":prop_type}

    headers = {
        'x-rapidapi-key': "3098984b99msh9a120a8ade10e53p1ebdd1jsn7703d6e155d0",
        'x-rapidapi-host': "realtor.p.rapidapi.com"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)

    return response.json() # json format

In [5]:
# Once pick state and home type throw it through this function to get the proper readable format
def process_list_for_sale_response(response_json):
    """
    Process the list for sale API response.

    Convert each listing to a dataframe, append to a list, and concatenate to one dataframe.

    Parameters
    ----------
    @response_json [dictionary]: API response for list for sale

    Returns
    -------
    [dataframe] Dataframe of all list for sale responses

    """

    # empty dataframe
    dataframe_list = []

    # iterate through each for sale listing
    for l in response_json['properties']:

        # convert each listing to dataframe
        _temp_df = pd.DataFrame.from_dict(l, orient='index').T

        # append to dataframe list for all listings
        dataframe_list.append(_temp_df)

    # concatenate all dataframes, for missing col values enter null value
    return pd.concat(dataframe_list, axis=0, ignore_index=True, sort=False)

In [6]:
def make_house_data_clean(df_properties_for_sale_raw):
    df_properties_for_sale_raw = pd.DataFrame(df_properties_for_sale_raw)# transfer into dataframe
    df_pol_ps = df_properties_for_sale_raw['lot_size'].apply(pd.Series) # make this int column non interable into series then 
    df_properties_for_sale_raw['size'] = df_pol_ps['size'] # then make it into column 

    # pick out only the columns we want  and rename them 
    list2 = ['property_id', 'listing_id','address','prop_status','price', 'baths_full', 'baths', 'beds', 'building_size','last_update', 'lot_size','baths_half', 'size']
    result = df_properties_for_sale_raw[list2]
    result.columns = ['property_id', 'listing_id','address','prop_status','price', 'baths_full', 'baths', 'beds', 'building_size','last_update', 'lot_size','baths_half', 'lot_size']
    return result

In [7]:
def get_address_and_buildingsize(result):
    address = pd.json_normalize(result['address'])
    building_size = pd.json_normalize(result['building_size'])
    building_size.columns =['building_size','building_inside_units']
    result2 = pd.concat([address,building_size], axis=1)
    result3 = pd.concat([df,result2], axis=1)
    return result3


In [8]:
# Variables 
city = "myrtle beach"
state = "SC"
prop_type = "single_family"

property_list_for_sale_response = api_property_list_for_sale(
                                                             city=city, 
                                                             state=state, 
                                                             prop_type=prop_type,
                                                             limit=200)

In [9]:
df_properties_for_sale_raw = process_list_for_sale_response(response_json=property_list_for_sale_response)


In [241]:
df = make_house_data_clean(df_properties_for_sale_raw)
#list(df.columns)
df

,property_id,listing_id,address,prop_status,price,baths_full,baths,beds,building_size,last_update,lot_size,baths_half,lot_size
0,M9410361591,2929615304,"{'city': 'Myrtle Beach', 'line': '533 Ambrosia...",for_sale,235000,2,2,3,"{'size': 1449, 'units': 'sqft'}",2021-05-22T16:35:18Z,"{'size': 10890, 'units': 'sqft'}",NaN,10890.0
1,M9090577838,2929605773,"{'city': 'Myrtle Beach', 'line': '2185 Sanibel...",for_sale,424900,3,3,4,"{'size': 1725, 'units': 'sqft'}",2021-05-22T04:15:29Z,NaN,NaN,NaN
2,M9155203410,2929592057,"{'city': 'Myrtle Beach', 'line': '9035 Gatewic...",for_sale,249900,2,2,3,"{'size': 1394, 'units': 'sqft'}",2021-05-21T20:51:17Z,"{'size': 7841, 'units': 'sqft'}",NaN,7841.0
3,M9416395684,2929590343,"{'city': 'Myrtle Beach', 'line': '650 Elgin Ct...",for_sale,394900,3,4,4,"{'size': 2624, 'units': 'sqft'}",2021-05-22T00:31:43Z,"{'size': 15682, 'units': 'sqft'}",1,15682.0
4,M5723306696,2929587613,"{'city': 'Myrtle Beach', 'line': '2534 Goldfin...",for_sale,400000,2,2,3,"{'size': 2148, 'units': 'sqft'}",2021-05-21T19:44:11Z,"{'size': 13068, 'units': 'sqft'}",NaN,13068.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,M9453125981,2929083403,"{'city': 'Myrtle Beach', 'line': '1424 Brighto...",for_sale,255000,2,3,4,"{'size': 1850, 'units': 'sqft'}",2021-05-22T13:37:37Z,"{'size': 6098, 'units': 'sqft'}",1,6098.0
196,M9549522450,2929083084,"{'city': 'Myrtle Beach', 'line': '2653 Corn Pi...",for_sale,249000,2,2,3,"{'size': 1464, 'units': 'sqft'}",2021-05-10T21:58:38Z,"{'size': 19602, 'units': 'sqft'}",NaN,19602.0
197,M9698682059,2929082893,"{'city': 'Myrtle Beach', 'line': '6001-1454 S ...",for_sale,237000,3,3,4,"{'size': 1610, 'units': 'sqft'}",2021-05-17T15:11:36Z,"{'size': 1742, 'units': 'sqft'}",NaN,1742.0
198,M9517132030,2929080051,"{'city': 'Myrtle Beach', 'line': '3556 Lightho...",for_sale,198500,2,2,2,"{'size': 1076, 'units': 'sqft'}",2021-05-09T18:04:01Z,"{'size': 8712, 'units': 'sqft'}",NaN,8712.0


In [254]:
df

,property_id,listing_id,address,prop_status,price,baths_full,baths,beds,building_size,last_update,lot_size,baths_half,lot_size
0,M9410361591,2929615304,"{'city': 'Myrtle Beach', 'line': '533 Ambrosia...",for_sale,235000,2,2,3,"{'size': 1449, 'units': 'sqft'}",2021-05-22T16:35:18Z,"{'size': 10890, 'units': 'sqft'}",NaN,10890.0
1,M9090577838,2929605773,"{'city': 'Myrtle Beach', 'line': '2185 Sanibel...",for_sale,424900,3,3,4,"{'size': 1725, 'units': 'sqft'}",2021-05-22T04:15:29Z,NaN,NaN,NaN
2,M9155203410,2929592057,"{'city': 'Myrtle Beach', 'line': '9035 Gatewic...",for_sale,249900,2,2,3,"{'size': 1394, 'units': 'sqft'}",2021-05-21T20:51:17Z,"{'size': 7841, 'units': 'sqft'}",NaN,7841.0
3,M9416395684,2929590343,"{'city': 'Myrtle Beach', 'line': '650 Elgin Ct...",for_sale,394900,3,4,4,"{'size': 2624, 'units': 'sqft'}",2021-05-22T00:31:43Z,"{'size': 15682, 'units': 'sqft'}",1,15682.0
4,M5723306696,2929587613,"{'city': 'Myrtle Beach', 'line': '2534 Goldfin...",for_sale,400000,2,2,3,"{'size': 2148, 'units': 'sqft'}",2021-05-21T19:44:11Z,"{'size': 13068, 'units': 'sqft'}",NaN,13068.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,M9453125981,2929083403,"{'city': 'Myrtle Beach', 'line': '1424 Brighto...",for_sale,255000,2,3,4,"{'size': 1850, 'units': 'sqft'}",2021-05-22T13:37:37Z,"{'size': 6098, 'units': 'sqft'}",1,6098.0
196,M9549522450,2929083084,"{'city': 'Myrtle Beach', 'line': '2653 Corn Pi...",for_sale,249000,2,2,3,"{'size': 1464, 'units': 'sqft'}",2021-05-10T21:58:38Z,"{'size': 19602, 'units': 'sqft'}",NaN,19602.0
197,M9698682059,2929082893,"{'city': 'Myrtle Beach', 'line': '6001-1454 S ...",for_sale,237000,3,3,4,"{'size': 1610, 'units': 'sqft'}",2021-05-17T15:11:36Z,"{'size': 1742, 'units': 'sqft'}",NaN,1742.0
198,M9517132030,2929080051,"{'city': 'Myrtle Beach', 'line': '3556 Lightho...",for_sale,198500,2,2,2,"{'size': 1076, 'units': 'sqft'}",2021-05-09T18:04:01Z,"{'size': 8712, 'units': 'sqft'}",NaN,8712.0


In [251]:
df['price'].max() # 1949000
df['price'].min() # 34900
df['price'].mean() # 404515.635


404515.635

# Vizualize the points on a map

In [243]:
list(var2.columns)

['property_id',
 'listing_id',
 'address',
 'prop_status',
 'price',
 'baths_full',
 'baths',
 'beds',
 'building_size',
 'last_update',
 'lot_size',
 'baths_half',
 'lot_size',
 'city',
 'line',
 'postal_code',
 'state_code',
 'state',
 'county',
 'fips_code',
 'county_needed_for_uniq',
 'lat',
 'lon',
 'neighborhood_name',
 'time_zone',
 'is_approximate',
 'building_size',
 'building_inside_units']

In [242]:
# expand the address and building size columns 
var2 = get_address_and_buildingsize(df)
#var2['line']


In [266]:
lon = list(var2["lon"]) # make longitude adn lattitude into list
lat = list(var2["lat"])
price = list(var2["price"]) # This normally needs to be a string!!!!!!!!!

# Color changing function 
def color_producer(prices):
    if prices <= 20000:
        return 'lightgreen'
    elif 20001 <= prices < 75000:
        return 'green'
    elif 75001 <= prices < 100000:
        return 'darkgreen'
    elif 100001 <= prices < 200000:
        return 'lightblue'
    elif 200001 <= prices < 250000:
        return 'blue'
    elif 250001 <= prices < 300000:
        return 'darkblue'
    elif 300001 <= prices < 350000:
        return 'orange'
    elif 350001 <= prices < 400000:
        return 'lightred'
    else: 
        return 'red'
# 'pink', 'darkblue', 'red', 'purple', 'darkred', 'cadetblue', 'lightred', 'green', 'darkgreen', 'beige', 'lightgreen', 'orange', 'white', 'gray', 'darkpurple', 'blue', 'lightgray', 'black', 'lightblue'}.
# initialize map
map = folium.Map(location = [38.28,-99.09],zoom_start = 6,TileLayer="Mapbox Bright")
#make map fg variable 
fg = folium.FeatureGroup(name="My map")

for lt, ln, pr in zip(lat,lon,price):
        fg.add_child(folium.Marker(location =[lt,ln],popup=str(pr) + " dollars" ,icon =folium.Icon(color=color_producer(pr))))

map.add_child(fg)
map.save("Map10.html")
map

# Loop through dataframe street name using selenium and Melisa.com

In [169]:
# Loop through all of the columns then get the names you need to make the new url to throw in the selenium code.. 
for i in range(0,len(var2)):   
    try:
        street = var2['line'][i]                                 # get the street name
        result = ''.join([i for i in street if not i.isdigit()]) # search for digits and replace them when searching for the street
        result =result.strip()                                   # remove leading and trailing spaces
        street = result.replace(" ", "+")                        # replace spances with plus ( thats how the website likes it)

        # Get state name then
        state = var2['state'][i]
        state = us_state_abbrev[state]   # convert full name to abbriviation

        # get postal code.. 
        zipcode = var2['postal_code'][i]

        # get city and then replace the space with plus
        city = var2['city'][i]
        city = city.replace(" ", "+")
        #print(street + " " + state + " " + zipcode +" " + city) VERIFY!!!

        # Get the url for the selenium loop... 
        link_to_neighbors = 'https://www.melissa.com/v2/lookups/addresssearch/?number=&street={0}&city={1}&state={2}&zip={3}&freeForm='.format(street,city,state,zipcode)

        # initiate crome driver
        c = webdriver.ChromeOptions()

        # get the path to the chrome driver 
        driver = webdriver.Chrome(executable_path='C:/Users/btindol/OneDrive - Stryker/ChromeDriver/chromedriver.exe')#,options=c)

        # wait a bit
        driver.implicitly_wait(0.5)

        # use the url that got in above code 
        driver.get(link_to_neighbors) # use the url that was created before... 

        # get past the pop up alert..
        WebDriverWait(driver, 10).until(EC.alert_is_present())

        # click out of it
        driver.switch_to.alert.accept()

        # wait a bit 
        driver.implicitly_wait(3)

        # grab email to input eamil for user name 
        emailfield= driver.find_element_by_id("ctl00_ContentPlaceHolder1_Signin1_txtEmail")

        # grab password to put in pass
        passwordmessagefield = driver.find_element_by_id("ctl00_ContentPlaceHolder1_Signin1_txtPassword")

        # put in email
        emailfield.send_keys(email)

        # put in password
        passwordmessagefield.send_keys(passwordz)

        # submit button 
        submitbutton = driver.find_element_by_id("ctl00_ContentPlaceHolder1_Signin1_btnLogin")
        submitbutton.click() # send click t osubmit

        # get the excel file on the page and download.. 
        excelfile = driver.find_element_by_xpath('//*[@id="content"]/section[3]/div/div/div/span/div/div[3]/input[3]')
        excelfile.click()
    except:
        pass

In [192]:
import os
direct = 'C:\\Users\\btindol\\Downloads\\'

listz = []
listnames = []
for filename in os.listdir('C:\\Users\\btindol\\Downloads'):
    if filename.endswith(".xlsx"):
        #print(filename)
        var = direct +  filename
        print(var)
        #do your operation
        data_df = pd.read_excel(var)#, sheet_name=None)
        listnameorder=data_df['Address'][0]
        listnames.append(listnameorder)
        data_df = pd.DataFrame(data_df)
        listz.append(data_df)
        #os.remove(data_df)
listnames

C:\Users\btindol\Downloads\address_202105221739.xlsx
C:\Users\btindol\Downloads\address_202105221740 (1).xlsx
C:\Users\btindol\Downloads\address_202105221740 (2).xlsx
C:\Users\btindol\Downloads\address_202105221740 (3).xlsx
C:\Users\btindol\Downloads\address_202105221740.xlsx
C:\Users\btindol\Downloads\address_202105221741 (1).xlsx
C:\Users\btindol\Downloads\address_202105221741.xlsx
C:\Users\btindol\Downloads\address_202105221745 (1).xlsx
C:\Users\btindol\Downloads\address_202105221745.xlsx
C:\Users\btindol\Downloads\address_202105221746 (1).xlsx
C:\Users\btindol\Downloads\address_202105221746 (2).xlsx
C:\Users\btindol\Downloads\address_202105221746 (3).xlsx
C:\Users\btindol\Downloads\address_202105221746 (4).xlsx
C:\Users\btindol\Downloads\address_202105221746.xlsx
C:\Users\btindol\Downloads\address_202105221747.xlsx
C:\Users\btindol\Downloads\address_202105221748 (1).xlsx
C:\Users\btindol\Downloads\address_202105221748 (2).xlsx
C:\Users\btindol\Downloads\address_202105221748.xlsx
C:

['500 Ambrosia Loop',
 '9000 Gatewick Ct',
 '601 Elgin Ct',
 '2350 Goldfinch Dr',
 '2130 Sanibel Ct',
 '2700 Canvasback Trl',
 '6303 Porcher Dr',
 '2130 Sanibel Ct',
 '500 Ambrosia Loop',
 '601 Elgin Ct',
 '2350 Goldfinch Dr',
 '6303 Porcher Dr',
 '2700 Canvasback Trl',
 '9000 Gatewick Ct',
 '1509 Harlow Ct',
 '100 Southbury Dr',
 '2602 Temperance Dr',
 '700 Cane Pole Ct',
 '4854 Westwind Dr',
 '500 Oak Circle Dr',
 '2554 Wild Game Trl',
 '561 Indigo Bay Cir',
 '1100 Bluffton Ct',
 '3800 Pine Lake Dr',
 '401 Pennington Loop',
 '101 Queens Harbour Blvd',
 '100 Barclay Dr',
 '100 Carnaby Loop',
 '300 Harbour Reef Dr',
 '5001 Big Bear Ct',
 '401 Mooreland Dr',
 '1 Plantation Rd',
 '1100 King St',
 '300 Troon Ln',
 '2101 Birchwood Cir',
 '2047 Forestbrook Rd',
 '1406 Saint Thomas Cir',
 '5004 Wickalow Way',
 '1 San Marks Ct',
 '2000 Copper Creek Ct',
 '5508 Springs Ave',
 '1406 Saint Thomas Cir',
 '5701 Longleaf Dr',
 '104 Whipple Run Loop',
 '200 Black Pearl Way',
 '300 Esher Ct',
 '300 C

In [194]:
# matching file names to the addresses.
dfzzz= pd.concat(listz)
dfzzz

,HouseNumber,Street,StreetSuffix,City,State,Zip,StreetName,Address,StateName,Zip4,AddressType,RBDI,Active,Fips,County,ResultCode,MAK
0,500,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,500 Ambrosia Loop,South Carolina,NaN,,Residential,Yes,45051.0,horry,AS03,1442484286
1,501,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,501 Ambrosia Loop,South Carolina,7054.0,Street,Residential,Yes,45051.0,horry,AS01,1078919714
2,505,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,505 Ambrosia Loop,South Carolina,7054.0,Street,Residential,Yes,45051.0,horry,AS01,6241258760
3,509,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,509 Ambrosia Loop,South Carolina,7054.0,Street,Residential,Yes,45051.0,horry,AS01,9622347032
4,513,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,513 Ambrosia Loop,South Carolina,7054.0,Street,Residential,Yes,45051.0,horry,AS01,1328098783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,3546,Lighthouse,Way,Myrtle Beach,SC,29577,NaN,3546 Lighthouse Way,South Carolina,5937.0,Street,Residential,Yes,45051.0,horry,AS01,8700486088
16,3549,Lighthouse,Way,Myrtle Beach,SC,29577,NaN,3549 Lighthouse Way,South Carolina,5938.0,Street,Residential,Yes,45051.0,horry,AS01,2654063784
17,3553,Lighthouse,Way,Myrtle Beach,SC,29577,NaN,3553 Lighthouse Way,South Carolina,5938.0,Street,Residential,Yes,45051.0,horry,AS01,6341010639
18,3556,Lighthouse,Way,Myrtle Beach,SC,29577,NaN,3556 Lighthouse Way,South Carolina,5937.0,Street,Residential,Yes,45051.0,horry,AS01,3432408415


In [204]:
# Here you want to take the first name of the street without number so 123 house lane would be house streeet suffix lane so just put in house
def get_houses_on_street(dataframe,street):
    try:
        final= dataframe[dataframe['Street']== street]
    except:
        print("no address")
    return final

In [207]:
var = get_houses_on_street(dfzzz,"Ambrosia")
var

,HouseNumber,Street,StreetSuffix,City,State,Zip,StreetName,Address,StateName,Zip4,AddressType,RBDI,Active,Fips,County,ResultCode,MAK
0,500,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,500 Ambrosia Loop,South Carolina,NaN,,Residential,Yes,45051.0,horry,AS03,1442484286
1,501,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,501 Ambrosia Loop,South Carolina,7054.0,Street,Residential,Yes,45051.0,horry,AS01,1078919714
2,505,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,505 Ambrosia Loop,South Carolina,7054.0,Street,Residential,Yes,45051.0,horry,AS01,6241258760
3,509,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,509 Ambrosia Loop,South Carolina,7054.0,Street,Residential,Yes,45051.0,horry,AS01,9622347032
4,513,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,513 Ambrosia Loop,South Carolina,7054.0,Street,Residential,Yes,45051.0,horry,AS01,1328098783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,577,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,577 Ambrosia Loop,South Carolina,7054.0,Street,Residential,Yes,45051.0,horry,AS01,4286786007
33,578,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,578 Ambrosia Loop,South Carolina,7053.0,Street,Residential,Yes,45051.0,horry,AS01,3108752463
34,581,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,581 Ambrosia Loop,South Carolina,7054.0,Street,Residential,Yes,45051.0,horry,AS01,4352165450
35,582,Ambrosia,Loop,Myrtle Beach,SC,29579,NaN,582 Ambrosia Loop,South Carolina,7053.0,Street,Residential,Yes,45051.0,horry,AS01,4685463988


In [215]:
len(var)

74

# Use zillow to get data

In [208]:
zillow_data = ZillowWrapper("X1-ZWz1fjckjdd8gb_a2eph")

In [217]:
streetlist =[]
zestimateslist = []
bedroomlist = []
bathroomlist = []
lastsoldpricelist = []
valuationrangehighlist = []
valuationrangelowlist = []
rentzestimateamountlist = []

for i in range(0,len(var)):
    try:
        address = var['Address'][i]
        zipcode =var['Zip'][i]
        deep_search_response = zillow_data.get_deep_search_results(address, zipcode, True)
        result = GetDeepSearchResults(deep_search_response)
        streetlist.append(result.street)
        zestimateslist.append(result.zestimate_amount)
        bedroomlist.append(result.bedrooms)
        bathroomlist.append(result.bathrooms)
        lastsoldpricelist.append(result.last_sold_price)
        valuationrangehighlist.append(result.zestimate_valuation_range_high)
        valuationrangelowlist.append(result.zestimate_valuation_range_low)
        rentzestimateamountlist.append(result.rentzestimate_amount)
        #print(i)
    except:
        #print("Error" + i)
        continue

In [221]:
street_dataframe = pd.DataFrame(
    {'streetz': streetlist,
     'zestimates': zestimateslist,
     'bedrooms': bedroomlist,
     'bathrooms': bathroomlist,
     'lastsoldprice': lastsoldpricelist,
     'valuationrangehigh': valuationrangehighlist,
     'valuationrangelowlist': valuationrangelowlist,
     'rentzestimateamountlist': rentzestimateamountlist
    })
street_dataframe

,streetz,zestimates,bedrooms,bathrooms,lastsoldprice,valuationrangehigh,valuationrangelowlist,rentzestimateamountlist
0,500 Ambrosia Loop,286541,3,2.5,252781,300868,269349,1750
1,501 Ambrosia Loop,220625,3,2.0,193000,244894,198563,1500
2,505 Ambrosia Loop,224309,3,2.0,177327,235524,213094,1499
3,509 Ambrosia Loop,241014,3,2.0,205030,253065,228963,1700
4,513 Ambrosia Loop,226121,3,2.0,175964,237427,214815,1500
5,517 Ambrosia Loop,236986,3,2.0,137000,253575,225137,1575
6,520 Ambrosia Loop,230632,3,2.0,196423,244470,219100,1600
7,521 Ambrosia Loop,230159,3,2.0,186291,241667,218651,1600
8,525 Ambrosia Loop,223431,3,2.0,156000,234603,212259,1500
9,529 Ambrosia Loop,216830,3,2.0,197000,236345,197315,1600


In [238]:
# What quantile is that house in .. 
import pandas as pd
import numpy as np
street_dataframe['zestimates'].dtypes
street_dataframe["zestimates"] = pd.to_numeric(street_dataframe["zestimates"])
street_dataframe['Quantile_rank'] = pd.qcut(street_dataframe['zestimates'], 10,labels = False)
street_dataframe

,streetz,zestimates,bedrooms,bathrooms,lastsoldprice,valuationrangehigh,valuationrangelowlist,rentzestimateamountlist,Quantile_rank
0,500 Ambrosia Loop,286541,3,2.5,252781,300868,269349,1750,9
1,501 Ambrosia Loop,220625,3,2.0,193000,244894,198563,1500,1
2,505 Ambrosia Loop,224309,3,2.0,177327,235524,213094,1499,4
3,509 Ambrosia Loop,241014,3,2.0,205030,253065,228963,1700,8
4,513 Ambrosia Loop,226121,3,2.0,175964,237427,214815,1500,4
5,517 Ambrosia Loop,236986,3,2.0,137000,253575,225137,1575,8
6,520 Ambrosia Loop,230632,3,2.0,196423,244470,219100,1600,5
7,521 Ambrosia Loop,230159,3,2.0,186291,241667,218651,1600,5
8,525 Ambrosia Loop,223431,3,2.0,156000,234603,212259,1500,3
9,529 Ambrosia Loop,216830,3,2.0,197000,236345,197315,1600,1
